#### Import all required libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error , mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasRegressor

## Task 1 : Load the dataset and perform preliminary EDA with key observations and insights- (weightage - 20 marks)

T1.1 :                                                    
a)Load the california_housing dataset as df outside of the function as df                                  
b)inside the load_california_housing() function, again load the California_housing dataset and return first five rows and shape of the dataframe 

In [ ]:
def load_california_housing(dataset_location):
    df= None
    #code starts here
    df = pd.read_csv(dataset_location)
    #code ends here
    return df

In [ ]:
# store the result of the dataset
dataset_location = 'california_housing.csv'
df = load_california_housing(dataset_location)
df

In [ ]:
def df_head_shape(df):
    head ,shape =  None , None
    #code starts here
    head = df.head()
    shape = df.shape
    #code ends here
    return head,shape

In [ ]:
df_head_shape(df)

In [ ]:
def missing_values(df):
    x =  None
    #code starts here
    df.drop(columns=['id'],inplace=True)
    x = df.isnull().sum()
    # code ends here
    return x

In [ ]:
def find_iqr(df, col):
    iqr_value = None
    # code starts here
    Q3 = np.quantile(df[col], 0.75)
    Q1 = np.quantile(df[col], 0.25)
    iqr_value = Q3 - Q1
    # code ends here
    return iqr_value

In [ ]:
def remove_outliers_iqr(df, columns):
    for column in columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

In [ ]:
def calculate_correlation_coefficients(df_clean):
    correlation_matrix,highest_correlation_coefficient = None , None
    #code starts here
    correlation_matrix = df_clean.corr() 
    highest_correlation_coefficient = correlation_matrix["MedHouseVal"].drop("MedHouseVal").abs().max()
    #code ends here
    return correlation_matrix, highest_correlation_coefficient

In [ ]:
def Normalize(df_clean):
    df_scaled = None
    #code starts here
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df_clean.drop(columns=['MedHouseVal']))
    df_scaled = pd.DataFrame(df_scaled, columns=df_clean.columns[:-1],index= df_clean.index)
    df_scaled['MedHouseVal'] = df_clean['MedHouseVal']
    # code ends here

    return df_scaled

In [ ]:
def train_split(df_scaled):
    x_train, x_test, y_train, y_test = None , None , None , None
    # code starts here
    # divide the data into input and output
    x = df_scaled.drop(columns=['MedHouseVal'])
    y = df_scaled['MedHouseVal']
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    # code ends here
    return x_train, x_test, y_train, y_test

In [ ]:
def get_highest_vif(df_scaled):
     highest_vif_column_name, sorted_df = None , None
     #code starts here 
     x = df_scaled.drop(columns=['MedHouseVal'])

     vif_df = pd.DataFrame()

     # Add feature names to the DataFrame
     vif_df["feature"] = x.columns

     # Calculate VIF for each feature
     vif_df["VIF"] = [variance_inflation_factor(x.values, i)
                         for i in range(len(x.columns))]

     # Filter columns with VIF greater than 5
     vif_df = vif_df[vif_df["VIF"] <= 5]

     # Sort the DataFrame by VIF in descending order
     sorted_df = vif_df.sort_values(by='VIF', ascending=False)

     # Get the name of the column with the highest VIF (the first row after sorting)
     highest_vif_column_name = sorted_df.iloc[0]['feature']

    
     ## code ends here   
     return highest_vif_column_name, sorted_df


In [ ]:
def create_model(x_train):
    model = None
    #code starts here
    input_shape=(x_train.shape[1],)
    model = Sequential([
        Dense(128, activation='relu', input_shape=input_shape),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1)])
    #code ends here
    return model

In [ ]:
#template
def compile_and_fit_model(model,x_train, y_train):
    #code starts here
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x_train, y_train, epochs=50, batch_size=32, verbose=0)
    #code ends here
    return model

In [ ]:
def evaluate_model(model, x_test, y_test):
    mse,rmse,mae,r_squared,mape,adj_r_squared = None , None , None , None, None , None
    #code starts here
    y_pred = model.predict(x_test,verbose = 0)
    y_test = np.array(y_test).flatten()

    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    r_squared = r2_score(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test,y_pred)

    
    # Adjusted R-squared
    n = len(x_test)
    p = x_test.shape[1]
    adj_r_squared = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)

    #code ends here
    return mse,rmse,mae,r_squared,mape,adj_r_squared